In [1]:
import numpy as np
import wordsegment as ws
import pickle
import re

In [19]:
import time, sys

In [20]:
def update_progress(progress):
    barLength = 100 # Modify this to change the length of the progress bar
    status = ""
    if isinstance(progress, int):
        progress = float(progress)
    if not isinstance(progress, float):
        progress = 0
        status = "error: progress var must be float\r\n"
    if progress < 0:
        progress = 0
        status = "Halt...\r\n"
    if progress >= 1:
        progress = 1
        status = "Done...\r\n"
    block = int(round(barLength*progress))
    text = "\rPercent: [{0}] {1}% {2}".format( "#"*block + "-"*(barLength-block), progress*100, status)
    sys.stdout.write(text)
    sys.stdout.flush()

Loading data for processing

In [11]:
fp_train = open("./datasets/train/SemEval2018-T3-train-taskA_emoji.txt",'r',encoding="utf-8")
ws.load()

The code in below cell is function for obtaining text from a single tweet.

In [12]:
def getDataFromTweet(sample):
    idx, label, tweet = sample.split('\t')
    hashtags = [i[1:] for i in tweet.split() if i.startswith("#")]
    hashtag_words = []
    text_words = [re.sub(r'[^\w\s]','',i).lower() for i in tweet.split() if not i.startswith("#")]
    for hashtag in hashtags:
        hashtag_words.extend(ws.segment(hashtag))
    return idx, label, tweet, text_words, hashtag_words

This function reads all data from file and stores in lists

In [13]:
def readData(fp):
    samples = fp.read().strip().split('\n')
    samples = samples[1:]
    idxs = []
    tweets = []
    tweet_text = []
    tweet_hashtags = []
    labels = []

    for sample in samples:
        idx, label, tweet, text_words, hashtag_words = getDataFromTweet(sample)
        idxs.append(idx)
        labels.append(label)
        tweets.append(tweet)
        tweet_text.append(text_words)
        tweet_hashtags.append(hashtag_words)
    
    return idxs, tweets, tweet_text, tweet_hashtags, labels

In [14]:
idxs, tweets, tweet_text, tweet_hashtags, labels = readData(fp_train)

In [15]:
print(len(tweets))

3834


In [16]:
def makeWordList(sent_list):
    wf = {}
    for sent in sent_list:
        for w in sent:
            if w in wf:
                wf[w] += 1
            else:
                wf[w] = 0
    wl = {}
    rwl = {}
    i = 0
    for w,f in wf.items():
        wl[w] = i
        rwl[i] = w
        i += 1
    wl['UNK'] = i
    return wl,rwl

In [17]:
word_list, rev_word_list = makeWordList(tweet_text+tweet_hashtags)

In [18]:
print(len(word_list))

12348


In [8]:
def mapWordToId(sent_contents,word_dict):
    T = []
    for sent in sent_contents:
        t = []
        for w in sent:
            if w in word_dict:
                t.append(word_dict[w])
            else:
                t.append(word_dict['UNK'])
        T.append(t)
    return T

In [9]:
tweet_text_id = mapWordToId(tweet_text,word_list)
tweet_hashtags_id = mapWordToId(tweet_hashtags,word_list)

In [10]:
print (tweet_text_id[0])
print (tweet_hashtags_id[0])

[1113, 5109, 2293, 10210, 3897, 1508, 2138, 98, 7623, 9741]
[8484, 5321, 6401, 2457]


In [23]:
def readWordEmb(word_dict, fname, embSize=100):
    print ("Reading word vectors from file")
    wv = []
    wl = []
    num_lines = sum(1 for line in open(fname, encoding="utf-8"))
    with open(fname, 'r',encoding="utf-8") as f:
        i = 1
        for line in f :
            vs = line.split()
            if len(vs) < 50 :
                continue
            vect = map(float, vs[1:])
            wv.append(vect)
            wl.append(vs[0])
            update_progress(i/num_lines)
            i += 1
    wordemb = []
    count = 0
    print ("Reading words from word list")
    for word, id in word_dict.items():
        i = 1
        if str(word) in wl:
            wordemb.append(wv[wl.index(str(word))])
        else:
            count += 1
            wordemb.append(np.random.rand(embSize))
        update_progress(i/len(word_dict))
        i += 1
    wordemb = np.asarray(wordemb, dtype='float32')
    print ("Number of unknown word in word embedding", count)
    return wordemb

In [ ]:
emb_file = "./glove/glove.twitter.27B.100d.txt"
wv = readWordEmb(word_list,emb_file)

Reading word vectors from file
Percent: [###################################################-------------------------------------------------] 51.113099636870615%     

Save all data in pickle file

In [ ]:
W_text = np.array(tweet_text_id)
W_hashtags = np.array(tweet_hashtags_id)
Y = np.zeros(len(labels), 2)
for i in range(len(labels)):
    Y[i][labels[i]] = 1

with open('./datasets/taskA-train.pickle', 'wb') as handle:
    pickle.dump(W_text, handle)
    pickle.dump(W_hashtags, handle)
    pickle.dump(Y, handle)
    pickle.dump(wv, handle)
    pickle.dump(word_list, handle)
